In [1]:

import torch
from torchvision import datasets, transforms
from torch.utils import data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.colors import ListedColormap, BoundaryNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm

np.random.seed(42)

NUM_CLASSES = 9
BATCH_SIZE = 32

PATH_TO_TEST_DATASET = "data/CRC-VAL-HE-7K/"

from model import DeepCMorph

In [2]:
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")

In [4]:
torch.backends.cudnn.deterministic = True
device = get_device()

model = DeepCMorph(num_classes=NUM_CLASSES, freeze_classification_module=False)
model.load_weights(dataset="CRC")

model.to(device)
model.eval()

test_transforms = transforms.Compose([transforms.ToTensor()])
test_dataset = datasets.ImageFolder(PATH_TO_TEST_DATASET, transform=test_transforms)
test_dataloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=False, drop_last=False)
TEST_SIZE = len(test_dataloader.dataset)
    

Model loaded, unexpected keys: []


In [5]:
tissue_labels = {
    0: 'adipose',
    1: 'background',
    2: 'debris',
    3: 'lymphocyte',
    4: 'mucus',
    5: 'smooth muscle',
    6: 'normal colon mucosa',
    7: 'cancer-associated stroma',
    8: 'colorectal adenocarcinoma epithelium' # tumor
}

cell_labels = {
    0: 'background',
    1: 'epithelial cell', 
    2: 'connective tissue cell', 
    3: 'lymphocyte', 
    4: 'plasma cell', 
    5: 'neutrophil',  
    6: 'eosinophil'
}

cell_colors = [
    'black',      # background
    'blue',       # epithelial cell
    'green',      # connective tissue cell
    'red',        # lymphocyte
    'yellow',     # plasma cell
    'purple',     # neutrophil
    'orange'      # eosinophil
]

In [9]:
#find class offsets in the dataset
class_offsets = {}
for i in range(NUM_CLASSES):
    class_offsets[i] = test_dataset.targets.index(i)
    
print(class_offsets)

{0: 0, 1: 1338, 2: 2185, 3: 2524, 4: 3158, 5: 4193, 6: 4785, 7: 5526, 8: 5947}


In [12]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import cv2

target_layers = [list(model.encoder.features.children())[-1]]
  
# pick image from each class
images = [test_dataset[class_offsets[i]] for i in range(NUM_CLASSES)]

# Construct the CAM object once, and then re-use it on many images.
with GradCAM(model=model, target_layers=target_layers) as cam:
  for sample in images:
    img = sample[0]
    target = sample[1]
    input_tensor = img.unsqueeze(0).to(device)
    targets = [ClassifierOutputTarget(target)]
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
    # In this example grayscale_cam has only one image in the batch:
    grayscale_cam = grayscale_cam[0, :]
    image = np.float32(img.permute(1, 2, 0))
    visualization = show_cam_on_image(image, grayscale_cam, use_rgb=False)
    cv2.imwrite(f'cam_{tissue_labels[target]}.jpg', visualization)
